# The Vanishing/Exploding Gradients Problems
## Glorot and He Initialization

In [1]:
import tensorflow as tf

dense = tf.keras.layers.Dense(50, activation="relu", kernel_initializer="he_normal")

In [2]:
he_avg_init = tf.keras.initializers.VarianceScaling(scale=2., mode="fan_avg", distribution="uniform")
dense = tf.keras.layers.Dense(50, activation="sigmoid", kernel_initializer=he_avg_init)

## Better Activation Functions
### Leaky ReLU

In [3]:
leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.2) # defaults to alpha=0.3

dense = tf.keras.layers.Dense(50, activation=leaky_relu, kernel_initializer="he_normal")

In [4]:
model = tf.keras.models.Sequential([
# [...] # more layers
tf.keras.layers.Dense(50, kernel_initializer="he_normal"), # no activation
tf.keras.layers.LeakyReLU(alpha=0.2), # activation as a separate layer
# [...] # more layers
])

### ELU and SELU
### GELU, Swish, and Mish
## Batch Normalization
### Implementing batch normalization with Keras

In [5]:
tf.keras.backend.clear_session()

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation="softmax")
])


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)              4

In [7]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [8]:
tf.keras.backend.clear_session()

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

## Gradient Clipping

In [9]:
optimizer = tf.keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer)

In [10]:
optimizer = tf.keras.optimizers.SGD(clipnorm=1.0)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer)

# Reusing Pretrained Layers
## Transfer Learning with Keras

In [11]:
import numpy as np

class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]
X_train, X_valid, X_test = X_train / 255, X_valid / 255, X_test / 255


pos_class_id = class_names.index("Pullover")
neg_class_id = class_names.index("T-shirt/top")

def split_dataset(X, y):
    y_for_B = (y == pos_class_id) | (y == neg_class_id)
    y_A = y[~y_for_B]
    y_B = (y[y_for_B] == pos_class_id).astype(np.float32)
    old_class_ids = list(set(range(10)) - set([neg_class_id, pos_class_id]))
    for old_class_id, new_class_id in zip(old_class_ids, range(8)):
        y_A[y_A == old_class_id] = new_class_id  # reorder class ids for A
    return ((X[~y_for_B], y_A), (X[y_for_B], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

tf.random.set_seed(42)

model_A = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(8, activation="softmax")
])

model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
                metrics=["accuracy"])
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                      validation_data=(X_valid_A, y_valid_A))
model_A.save("my_model_A")

Epoch 1/20
1376/1376 [==============================] - 3s 2ms/step - loss: 1.1080 - accuracy: 0.6623 - val_loss: 0.6812 - val_accuracy: 0.7889
Epoch 2/20
1376/1376 [==============================] - 2s 2ms/step - loss: 0.5910 - accuracy: 0.8137 - val_loss: 0.5084 - val_accuracy: 0.8363
Epoch 3/20
1376/1376 [==============================] - 2s 2ms/step - loss: 0.4772 - accuracy: 0.8502 - val_loss: 0.4388 - val_accuracy: 0.8528
Epoch 4/20
1376/1376 [==============================] - 2s 2ms/step - loss: 0.4188 - accuracy: 0.8666 - val_loss: 0.3958 - val_accuracy: 0.8669
Epoch 5/20
1376/1376 [==============================] - 2s 2ms/step - loss: 0.3820 - accuracy: 0.8751 - val_loss: 0.3707 - val_accuracy: 0.8726
Epoch 6/20
1376/1376 [==============================] - 2s 2ms/step - loss: 0.3566 - accuracy: 0.8816 - val_loss: 0.3516 - val_accuracy: 0.8752
Epoch 7/20
1376/1376 [==============================] - 2s 2ms/step - loss: 0.3387 - accuracy: 0.8856 - val_loss: 0.3387 - val_accuracy:

In [12]:
tf.random.set_seed(42)
model_B = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model_B.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
                metrics=["accuracy"])
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))
model_B.evaluate(X_test_B, y_test_B)

Epoch 1/20
7/7 [==============================] - 1s 51ms/step - loss: 0.7249 - accuracy: 0.5550 - val_loss: 0.7285 - val_accuracy: 0.5143
Epoch 2/20
7/7 [==============================] - 0s 16ms/step - loss: 0.6928 - accuracy: 0.5550 - val_loss: 0.7014 - val_accuracy: 0.5143
Epoch 3/20
7/7 [==============================] - 0s 17ms/step - loss: 0.6704 - accuracy: 0.5550 - val_loss: 0.6833 - val_accuracy: 0.5143
Epoch 4/20
7/7 [==============================] - 0s 15ms/step - loss: 0.6531 - accuracy: 0.5600 - val_loss: 0.6664 - val_accuracy: 0.5153
Epoch 5/20
7/7 [==============================] - 0s 14ms/step - loss: 0.6379 - accuracy: 0.5600 - val_loss: 0.6495 - val_accuracy: 0.5223
Epoch 6/20
7/7 [==============================] - 0s 14ms/step - loss: 0.6227 - accuracy: 0.5700 - val_loss: 0.6342 - val_accuracy: 0.5351
Epoch 7/20
7/7 [==============================] - 0s 13ms/step - loss: 0.6081 - accuracy: 0.5950 - val_loss: 0.6201 - val_accuracy: 0.5618
Epoch 8/20
7/7 [===========

[0.49426114559173584, 0.9135000109672546]

In [13]:
model_A_clone = tf.keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

model_B_on_A = tf.keras.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(tf.keras.layers.Dense(1, activation="sigmoid"))

In [14]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [15]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer,
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4


7/7 [==============================] - 1s 43ms/step - loss: 1.6804 - accuracy: 0.4400 - val_loss: 1.0825 - val_accuracy: 0.4837
Epoch 2/4
7/7 [==============================] - 0s 14ms/step - loss: 0.8894 - accuracy: 0.4550 - val_loss: 0.6859 - val_accuracy: 0.5371
Epoch 3/4
7/7 [==============================] - 0s 13ms/step - loss: 0.6390 - accuracy: 0.6300 - val_loss: 0.6286 - val_accuracy: 0.6568
Epoch 4/4
7/7 [==============================] - 0s 13ms/step - loss: 0.5991 - accuracy: 0.7150 - val_loss: 0.6204 - val_accuracy: 0.6677
Epoch 1/16
7/7 [==============================] - 1s 42ms/step - loss: 0.5749 - accuracy: 0.7150 - val_loss: 0.5769 - val_accuracy: 0.7201
Epoch 2/16
7/7 [==============================] - 0s 14ms/step - loss: 0.5330 - accuracy: 0.8300 - val_loss: 0.5248 - val_accuracy: 0.8170
Epoch 3/16
7/7 [==============================] - 0s 13ms/step - loss: 0.4866 - accuracy: 0.8750 - val_loss: 0.4960 - val_accuracy: 0.8131
Epoch 4/16
7/7 [=========================

In [16]:
model_B_on_A.evaluate(X_test_B, y_test_B)

40/63 [==================>...........] - ETA: 0s - loss: 0.2988 - accuracy: 0.9359

63/63 [==============================] - 0s 1ms/step - loss: 0.2984 - accuracy: 0.9390


[0.2984112501144409, 0.9390000104904175]

# Faster Optimizers
## Momentum

In [17]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)

## Nesterov Accelerated Gradient

In [18]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)

## AdaGrad
## RMSProp

In [19]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)

## Adam

In [20]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

tf.keras.optimizers.Adam with tf.keras.optimizers.Nadam,
tf.keras.optimizers.Adamax, or
tf.keras.optimizers.experimental.AdamW.

# Learning Rate Scheduling

### Power scheduling

In [21]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, decay=1e-4)

### Exponential scheduling

In [22]:
def exponential_decay_fn(epoch):
    return 0.01 * 0.1 ** (epoch / 20)

def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 ** (epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(lr0=0.01, s=20)

In [23]:
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)
# history = model.fit(X_train, y_train, [...], callbacks=[lr_scheduler])

In [24]:
def exponential_decay_fn(epoch, lr):
    return lr * 0.1 ** (1 / 20)

### piecewise constant scheduling

In [25]:
def piecewise_constant_fn(epoch):
    if epoch < 5:
        return 0.01
    elif epoch < 15:
        return 0.005
    else:
        return 0.001

### performance scheduling

In [26]:
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
# history = model.fit(X_train, y_train, [...], callbacks=[lr_scheduler])

### example scheduling

In [27]:
import math

batch_size = 32
n_epochs = 25
n_steps = n_epochs * math.ceil(len(X_train) / batch_size)
scheduled_learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.01, decay_steps=n_steps, decay_rate=0.1)
optimizer = tf.keras.optimizers.SGD(learning_rate=scheduled_learning_rate)

# Avoiding Overfitting Through Regularization
## $ℓ_1$ and $ℓ_2$ Regularization

In [28]:
layer = tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal", 
                              kernel_regularizer=tf.keras.regularizers.l2(0.01))

In [29]:
from functools import partial

RegularizedDense = partial(tf.keras.layers.Dense, activation="relu", kernel_initializer="he_normal",
                           kernel_regularizer=tf.keras.regularizers.l2(0.01))

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(100),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])

## Dropout

In [30]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="relu",kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(10, activation="softmax")
])

## Monte Carlo (MC) Dropout

In [31]:
import numpy as np

y_probas = np.stack([model(X_test, training=True) for sample in range(100)])
y_proba = y_probas.mean(axis=0)

In [32]:
model.predict(X_test[:1]).round(3)

1/1 [==============================] - 0s 121ms/step


array([[0.066, 0.11 , 0.123, 0.136, 0.073, 0.127, 0.08 , 0.068, 0.076,
        0.14 ]], dtype=float32)

In [33]:
y_proba[0].round(3)

array([0.069, 0.102, 0.119, 0.146, 0.066, 0.118, 0.083, 0.054, 0.087,
       0.156], dtype=float32)

In [34]:
y_std = y_probas.std(axis=0)
y_std[0].round(3)

array([0.026, 0.037, 0.049, 0.067, 0.02 , 0.037, 0.041, 0.021, 0.046,
       0.059], dtype=float32)

In [35]:
y_pred = y_proba.argmax(axis=1)
accuracy = (y_pred == y_test).sum() / len(y_test)
accuracy

0.1764

In [36]:
class MCDropout(tf.keras.layers.Dropout):
    def call(self, inputs, training=False):
        return super().call(inputs, training=True)

## Max-Norm Regularization

In [37]:
dense = tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal",
                              kernel_constraint=tf.keras.constraints.max_norm(1.))